# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção: (caso o site esteja preparado para evitar esse tipo de utilizaçao do selenium)

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail pythonimpressionador@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

In [12]:
import pandas as pd

produtos_df = pd.read_excel('arquivos/buscas.xlsx')
display(produtos_df)
produtos_df.info()

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch 11,3000,3700
1,rtx 3060,zota galax,1900,2500


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Nome            2 non-null      object
 1   Termos banidos  2 non-null      object
 2   Preço mínimo    2 non-null      int64 
 3   Preço máximo    2 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 192.0+ bytes


In [13]:
from selenium import webdriver # Comandar navegador controlado
from selenium.webdriver.chrome.service import Service # responsável por iniciar e parar o chromedriver
from webdriver_manager.chrome import ChromeDriverManager # Gerenciar arquvio ChromeDriver

# Instalar arquivo temporário ChromeDrive a partir da versão atual do Chrome
servico = Service(ChromeDriverManager().install())

# Abrir navegador com tela maximizada
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')

# Criar navegador
nav = webdriver.Chrome(service=servico, options=options)

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:00<00:00, 36.9MB/s]


In [14]:
from selenium.webdriver.common.by import By # Selecionar elementos
from selenium.webdriver.common.keys import Keys # Utilizar comandos do teclado

# Criando Function Para Listar Valores Encontrados Google Shopping

def pesquisa_google(nav, nome_produto, nomes_banidos, preço_min, preço_max):

    # Entrar google shopping
    nav.get('https://shopping.google.com/?nord=1')

    # Pesquisar produto
    nav.find_element(By.XPATH, '//*[@id="REsRA"]').send_keys(nome_produto)
    nav.find_element(By.XPATH, '//*[@id="REsRA"]').send_keys(Keys.ENTER)

    # Filtrar Valor min
    nav.find_element(By.NAME, 'lower').send_keys(str(preço_min))
    # Filtrar Valor max
    nav.find_element(By.NAME, 'upper').send_keys(str(preço_max))
    # Aplicar filtro de valor
    nav.find_element(By.NAME, 'upper').send_keys(Keys.ENTER)
        
    # Listando elementos com informações do produto pesquisado
    info_produto = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')

    # Listando termos banidos separadamente
    termos_banidos = nomes_banidos.lower()
    termos_banidos = termos_banidos.split(' ')

    # Listando termos do nome do produto separadamente
    termos_produto = nome_produto.lower()
    termos_produto = termos_produto.split(' ')

    # Criando lista com os nome, preço e link do produto
    resultado_pesquisa = []
    for elemento in info_produto:
        
        # Selecionando elemento com nome do produto
        nome_produto = elemento.find_element(By.CLASS_NAME, 'tAxDx').text.lower()

        # Verificando termo banido
        produto_banido = False
        for termo in termos_banidos:
            if termo in nome_produto:
                produto_banido = True
        
        # Verificando se o nome do produto bate com o pesquisado
        produto_aceito = True
        for termo in termos_produto:
            if termo not in nome_produto:
                produto_aceito = False

        # Filtrando nome do produto
        if not produto_banido and produto_aceito:

            # Selecionando elemento com preço do produto
            preço = elemento.find_element(By.CLASS_NAME, 'a8Pemb').text

            # Selecionando elemento com link do produto
            elemento_link_filho = elemento.find_element(By.CLASS_NAME, 'aULzUe')
            elemento_link_pai = elemento_link_filho.find_element(By.XPATH, '..')
            link = elemento_link_pai.get_attribute('href')
            
            resultado_pesquisa.append((nome_produto, preço, link))

    return resultado_pesquisa


In [15]:
import time

# Criando Function Para Listar Valores Encontrados Buscape

def pesquisa_buscape(nav, nome_produto, nomes_banidos, preço_min, preço_max):

    # Entrar buscapé
    nav.get('https://www.buscape.com.br/')

    # Pesquisar produto
    nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(nome_produto)
    nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(Keys.ENTER)

    time.sleep(3)
  
    # Listando elementos com informações do produto pesquisado
    info_produto = nav.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard__1D3ve')

    # Listando termos banidos separadamente
    termos_banidos = nomes_banidos.lower()
    termos_banidos = termos_banidos.split(' ')

    # Listando termos do nome do produto separadamente
    termos_produto = nome_produto.lower()
    termos_produto = termos_produto.split(' ')

    # Criando lista com os nome, preço e link do produto
    resultado_pesquisa = []
    for elemento in info_produto:
        
        # Selecionando elemento com nome do produto
        nome_produto = elemento.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text.lower()

        # Verificando termo banido
        produto_banido = False
        for termo in termos_banidos:
            if termo in nome_produto:
                produto_banido = True
        
        # Verificando se o nome do produto bate com o pesquisado
        produto_aceito = True
        for termo in termos_produto:
            if termo not in nome_produto:
                produto_aceito = False

        # Filtrando nome do produto
        if not produto_banido and produto_aceito:

            # Selecionando elemento com preço do produto
            preço = elemento.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
            preço_format = preço.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preço_format = float(preço_format)

            # Filtrando Valores
            if preço_min <= preço_format <= preço_max:

                # Selecionando elemento com link do produto
                link = elemento.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb').get_attribute('href')
            
                resultado_pesquisa.append((nome_produto, preço, link))

    return resultado_pesquisa


In [16]:
# Criando df separados para cada produto
iphone_df = pd.DataFrame(index=None)
rtx_df = pd.DataFrame(index=None)

# Rodando functions para pesquisar no gshopping e no buscapé percorrendo os produtos da tabela inicial
for linha in produtos_df.index:
    nomes_banidos = produtos_df.loc[linha, 'Termos banidos']
    nome_produto = produtos_df.loc[linha, 'Nome']
    preço_max = produtos_df.loc[linha, 'Preço máximo']
    preço_min = produtos_df.loc[linha, 'Preço mínimo']

    gshop = pesquisa_google(nav, nome_produto, nomes_banidos, preço_min, preço_max)
    if gshop:
        gshop_df = pd.DataFrame(gshop, columns=['PRODUTO', 'PREÇO', 'LINK'])
        if 'iphone' in gshop[0][0]:
            iphone_df = pd.concat([iphone_df, gshop_df])
            # valores_encontrados = pd.concat([valores_encontrados, gshop_df])
        elif 'rtx' in gshop[0][0]:
            rtx_df = pd.concat([rtx_df, gshop_df])
    else:
        gshop_df = None

    buscape = pesquisa_buscape(nav, nome_produto, nomes_banidos, preço_min, preço_max)
    if buscape:
        buscape_df = pd.DataFrame(buscape, columns=['PRODUTO', 'PREÇO', 'LINK'])
        if 'iphone' in buscape[0][0]:
            iphone_df = pd.concat([iphone_df, buscape_df])
            # valores_encontrados = pd.concat([valores_encontrados, gshop_df])
        elif 'rtx' in buscape[0][0]:
            rtx_df = pd.concat([rtx_df, buscape_df])
        # valores_encontrados = pd.concat([valores_encontrados, buscape_df])
    else:
        buscape_df = None

,PRODUTO,PREÇO,LINK
0,"iphone 12 apple vermelho, 64gb","R$ 3.699,00",https://www.google.com/url?url=https://www.ipl...
1,usado: iphone 12 64gb azul muito bom - trocafo...,"R$ 3.140,72",https://www.google.com/url?url=https://www.mag...
2,"iphone 12 64gb roxo tela 6,1 4g câmera traseir...","R$ 3.349,00",https://www.google.com/url?url=https://www.car...
3,usado: iphone 12 64gb preto bom - trocafone,"R$ 3.019,00",https://www.google.com/url?url=https://www.cas...
4,usado: iphone 12 64gb branco bom - trocafone,"R$ 3.109,00",https://www.google.com/url?url=https://www.bus...
5,usado: iphone 12 64gb preto excelente - trocaf...,"R$ 3.167,12",https://www.google.com/url?url=https://www.mag...
6,iphone 12 64gb tela 6.1 sem carregador e fone ...,"R$ 3.198,00",https://www.google.com/url?url=https://www.bus...
7,vitrine iphone 12 verde 64gb,"R$ 3.199,99",https://www.google.com/url?url=https://www.car...
8,celular apple iphone 12 black 64gb vitrine/sem...,"R$ 3.379,90",https://www.google.com/url?url=https://www.car...
9,iphone 12 64gb|celular apple| usado,"R$ 3.015,00",https://www.google.com/url?url=https://www.enj...


,PRODUTO,PREÇO,LINK
0,placa de vídeo msi geforce rtx 3060 ventus 2x ...,"R$ 2.099,90",https://www.google.com/url?url=https://www.ama...
1,placa de vídeo geforce rtx 3060 v2 dual 12gb g...,"R$ 2.099,99",https://www.google.com/url?url=https://www.mag...
2,placa de vídeo gigabyte geforce rtx 3060 windf...,"R$ 2.324,07",https://www.google.com/url?url=https://www.dhc...
3,placa de vídeo gigabyte geforce rtx 3060 gamin...,"R$ 2.329,99",https://www.google.com/url?url=https://www.kab...
4,placa de vídeo nvidia geforce rtx 3060 verto 1...,"R$ 2.240,00",https://www.google.com/url?url=https://www.mer...
5,placa de vídeo msi geforce rtx 3060 gaming x 1...,"R$ 2.014,67",https://www.google.com/url?url=https://s.click...
6,placa de video colorful igame geforce rtx 3060...,"R$ 2.463,12",https://www.google.com/url?url=https://www.pon...
7,placa de vídeo rtx 3060 oc edition asus dual g...,"R$ 2.059,99",https://www.google.com/url?url=https://www.mag...
8,placa de vídeo rtx 3060 vision oc 12g gigabyte...,"R$ 2.079,90",https://www.google.com/url?url=https://www.all...
9,placa de video igame rtx 3060 ultra w oc 8gb-v...,"R$ 2.339,10",https://www.google.com/url?url=https://www.pon...


In [17]:
# Ordeando preços do menor para o maior

iphone_df = iphone_df.sort_values(['PREÇO'])
display(iphone_df)

rtx_df = rtx_df.sort_values(['PREÇO'])
display(rtx_df)

,PRODUTO,PREÇO,LINK
9,iphone 12 64gb|celular apple| usado,"R$ 3.015,00",https://www.google.com/url?url=https://www.enj...
3,usado: iphone 12 64gb preto bom - trocafone,"R$ 3.019,00",https://www.google.com/url?url=https://www.cas...
4,usado: iphone 12 64gb branco bom - trocafone,"R$ 3.109,00",https://www.google.com/url?url=https://www.bus...
26,apple iphone 12 product red 64gb,"R$ 3.123,00",https://www.google.com/url?url=https://doji.co...
1,usado: iphone 12 64gb azul muito bom - trocafo...,"R$ 3.140,72",https://www.google.com/url?url=https://www.mag...
5,usado: iphone 12 64gb preto excelente - trocaf...,"R$ 3.167,12",https://www.google.com/url?url=https://www.mag...
16,apple iphone 12 64gb 5g vitrine original com g...,"R$ 3.168,00",https://www.google.com/url?url=https://produto...
11,iphone 12 64gb tela 6.1 sem carregador e fone ...,"R$ 3.198,00",https://www.google.com/url?url=https://www.bus...
0,smartphone apple iphone 12 64gb câmera dupla,"R$ 3.198,00",https://www.buscape.com.br/celular/smartphone-...
6,iphone 12 64gb tela 6.1 sem carregador e fone ...,"R$ 3.198,00",https://www.google.com/url?url=https://www.bus...


,PRODUTO,PREÇO,LINK
37,nvidia | placa de video rtx 3060 m 6gb gamer n...,"R$ 1.915,00",https://www.google.com/url?url=https://www.enj...
43,placa de video msi geforce rtx 3060 ventus 2x ...,"R$ 1.928,90",https://www.google.com/url?url=https://www.cas...
28,placa de vídeo asus geforce tuf gaming rtx 306...,"R$ 1.945,99",https://www.google.com/url?url=https://www.tra...
13,placa de vídeo nvidia verto dual fan rtx 3060 ...,"R$ 1.953,92",https://www.google.com/url?url=https://www.kab...
36,placa de video inno3d geforce rtx 3060 twin x2...,"R$ 1.959,00",https://www.google.com/url?url=https://www.pic...
32,"msi-rtx3060 placa gráfica, 2x oc, 12g ventus, ...","R$ 1.960,32",https://www.google.com/url?url=https://s.click...
10,placa de vídeo gainward ghost geforce rtx 3060...,"R$ 1.965,64",https://www.google.com/url?url=https://www.hds...
1,placa de video nvidia geforce rtx 3060 12 gb g...,"R$ 1.968,00",https://www.buscape.com.br/placa-de-video/plac...
12,placa de vídeo superframe nvidia geforce rtx 3...,"R$ 1.979,90",https://www.google.com/url?url=https://www.ter...
22,placa de video inno3d geforce rtx 3060 twin x2...,"R$ 1.999,00",https://www.google.com/url?url=https://www.ter...


In [18]:
# Exportando df dos resultados

iphone_df.to_excel('arquivos/resultados/iphone.xlsx')
rtx_df.to_excel('arquivos/resultados/rtx.xlsx')

In [21]:
display(iphone_df[:5])

,PRODUTO,PREÇO,LINK
9,iphone 12 64gb|celular apple| usado,"R$ 3.015,00",https://www.google.com/url?url=https://www.enj...
3,usado: iphone 12 64gb preto bom - trocafone,"R$ 3.019,00",https://www.google.com/url?url=https://www.cas...
4,usado: iphone 12 64gb branco bom - trocafone,"R$ 3.109,00",https://www.google.com/url?url=https://www.bus...
26,apple iphone 12 product red 64gb,"R$ 3.123,00",https://www.google.com/url?url=https://doji.co...
1,usado: iphone 12 64gb azul muito bom - trocafo...,"R$ 3.140,72",https://www.google.com/url?url=https://www.mag...


In [19]:
# Envio de email

# enviar por e-mail o resultado da tabela
import win32com.client as win32

#verificando se existe alguma oferta dentro da tabela de ofertas
if len(iphone_df.index) > 0 and len(rtx_df.index) > 0:
    # vou enviar email
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'nicolarthur17+Teste@hotmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {iphone_df[:10].to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    """
    
    mail.Send()

nav.quit()  